In [32]:
!nvidia-smi

Sun Dec  5 16:07:18 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.44       Driver Version: 495.44       CUDA Version: 11.5     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA A40          Off  | 00000000:3D:00.0 Off |                    0 |
|  0%   34C    P8    19W / 300W |      2MiB / 45634MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

## This code creates an image attribute description model for bumble bees ##

In [1]:
# necessary installations
!pip install xlrd

In [2]:
!pip install openpyxl

In [3]:
# necessary imports
import pandas as pd

### Load and preprocess dataset - images and text description ###

In [4]:
# load text
data = pd.read_excel("../Beemachine_Experiments_Logs.xlsx", sheet_name=None)

In [5]:
data.keys()

dict_keys(['Logs', 'Validation results - MobileNetV', 'Semantic Information', 'Correspondence', 'Key', 'Affinis', 'Auricomus', 'Bifarius', 'Bimaculatus'])

In [6]:
data_affinis = data['Affinis']

In [7]:
data_affinis.rename(columns={'ID':'Image-ID', 'Anterior Scutum':'Anterior-Scutum', 'Inner-Alar Space':'Inner-Alar-Space'}, inplace=True)

In [8]:
data_affinis.set_index('Image-ID', inplace=True)

In [9]:
data_affinis.dropna(how='all', inplace=True)

In [10]:
data_affinis.head()

,Head,Anterior-Scutum,Inner-Alar-Space,Scutellum,T1,T2,T3,T4,T5,T6,Corbicular
Image-ID,,,,,,,,,,,
1,B,Y,B,Y,Y,Y / O,B,B,B,B,NaN
2,B,Y,B,Y,Y,B,Y,B,B,B,NaN
3,B,Y,B,Y,Y,Y / O,B,B,B,B,NaN
4,B,Y,B,Y,Y,Y,B,B,B,B,Yes
5,B,Y,B,Y,Y,Y / O,B,B,B,B,NaN


### Optimize for memory efficiency ###

In [11]:
data_affinis['Corbicular'].fillna("False", inplace=True)

In [12]:
data_affinis['Corbicular'] = data_affinis['Corbicular'].str.strip().str.lower().str.replace("yes", "True")
data_affinis['Corbicular'] = data_affinis['Corbicular'].str.strip().str.lower().str.replace("no", "False")

In [13]:
for column in data_affinis.columns:
  data_affinis[f"{column}"] = f"{column}: " + data_affinis[f"{column}"]

In [14]:
data_affinis.head()

,Head,Anterior-Scutum,Inner-Alar-Space,Scutellum,T1,T2,T3,T4,T5,T6,Corbicular
Image-ID,,,,,,,,,,,
1,Head: B,Anterior-Scutum: Y,Inner-Alar-Space: B,Scutellum: Y,T1 : Y,T2 : Y / O,T3: B,T4: B,T5: B,T6: B,Corbicular: false
2,Head: B,Anterior-Scutum: Y,Inner-Alar-Space: B,Scutellum: Y,T1 : Y,T2 : B,T3: Y,T4: B,T5: B,T6: B,Corbicular: false
3,Head: B,Anterior-Scutum: Y,Inner-Alar-Space: B,Scutellum: Y,T1 : Y,T2 : Y / O,T3: B,T4: B,T5: B,T6: B,Corbicular: false
4,Head: B,Anterior-Scutum: Y,Inner-Alar-Space: B,Scutellum: Y,T1 : Y,T2 : Y,T3: B,T4: B,T5: B,T6: B,Corbicular: true
5,Head: B,Anterior-Scutum: Y,Inner-Alar-Space: B,Scutellum: Y,T1 : Y,T2 : Y / O,T3: B,T4: B,T5: B,T6: B,Corbicular: false


In [15]:
bumble_bees_attribute_description = ''
ind = 0
for column in data_affinis.columns:
  if ind != len(data_affinis.columns) - 1:
    bumble_bees_attribute_description += data_affinis[f"{column}"] + ', '
  else:
    bumble_bees_attribute_description += data_affinis[f"{column}"]
  ind += 1

In [16]:
pd.options.display.max_colwidth = 1000

In [17]:
bumble_bees_attribute_description.head()

Image-ID
1     Head: B, Anterior-Scutum: Y, Inner-Alar-Space: B, Scutellum: Y, T1 : Y, T2 : Y / O, T3: B, T4: B, T5: B, T6: B, Corbicular: false
2         Head: B, Anterior-Scutum: Y, Inner-Alar-Space: B, Scutellum: Y, T1 : Y, T2 : B, T3: Y, T4: B, T5: B, T6: B, Corbicular: false
3     Head: B, Anterior-Scutum: Y, Inner-Alar-Space: B, Scutellum: Y, T1 : Y, T2 : Y / O, T3: B, T4: B, T5: B, T6: B, Corbicular: false
4          Head: B, Anterior-Scutum: Y, Inner-Alar-Space: B, Scutellum: Y, T1 : Y, T2 : Y, T3: B, T4: B, T5: B, T6: B, Corbicular: true
5    Head: B, Anterior-Scutum: Y, Inner-Alar-Space: B, Scutellum: Y, T1 : Y, T2 : Y / O, T3: B, T4: B, T5: B, T6: B , Corbicular: false
Name: Head, dtype: object

In [31]:
pd.Series(bumble_bees_attribute_description.keys())

0        1
1        2
2        3
3        4
4        5
      ... 
245    246
246    247
247    248
248    249
249    250
Name: Image-ID, Length: 250, dtype: int64

In [18]:
data_affinis.nunique()

Head                2
Anterior-Scutum     1
Inner-Alar-Space    2
Scutellum           1
T1                  1
T2                  3
T3                  2
T4                  1
T5                  1
T6                  2
Corbicular          2
dtype: int64

In [19]:
data_affinis.nunique(dropna=False)

Head                3
Anterior-Scutum     1
Inner-Alar-Space    3
Scutellum           2
T1                  2
T2                  4
T3                  3
T4                  2
T5                  2
T6                  3
Corbicular          2
dtype: int64

In [20]:
data_affinis.dtypes

Head                object
Anterior-Scutum     object
Inner-Alar-Space    object
Scutellum           object
T1                  object
T2                  object
T3                  object
T4                  object
T5                  object
T6                  object
Corbicular          object
dtype: object

In [21]:
data_affinis.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 250 entries, 1 to 250
Data columns (total 11 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   Head              249 non-null    object
 1   Anterior-Scutum   250 non-null    object
 2   Inner-Alar-Space  249 non-null    object
 3   Scutellum         249 non-null    object
 4   T1                248 non-null    object
 5   T2                249 non-null    object
 6   T3                248 non-null    object
 7   T4                248 non-null    object
 8   T5                247 non-null    object
 9   T6                247 non-null    object
 10  Corbicular        250 non-null    object
dtypes: object(11)
memory usage: 23.4+ KB


In [22]:
# load images